In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
from tqdm import tqdm
import pickle
import warnings
from run_logistic_simulations import *
import seaborn as sns

RANDOM_SEED = 58
rng = np.random.default_rng(RANDOM_SEED)

In [ ]:
# doses = [0.5, 1, 3, 5, 6] # From figure 6 and in units (mg/m^2 per day)
doses = [2.9444390, -2.1972246, -1.7346011, -0.7081851,  0.0000000] # From figure 6 and in units (mg/m^2 per day)
true_toxic_prob_s1 = (0.25, 0.3, 0.5, 0.6, 0.7) # Given by assignment instructions, scenario 1
true_toxic_prob_s2 = (0.01, 0.05, 0.2, 0.3, 0.5) # Given by assignment instructions, scenario 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
p_i = np.array([0.05, 0.10, 0.15, 0.33, 0.50])
# doses_transformed = (np.log(p_i / (1 - p_i)) - 3) / np.exp(1) 
doses_transformed = [0.5, 1, 3, 5, 6]
doses_transformed

In [ ]:
import pymc as pm

def sigmoid(x):
  return 1 / (1 + np.exp(-x))
    
with pm.Model() as logistic_regression_model:
    x = pm.ConstantData("doses", doses_transformed, dims="observation")
    # priors
    beta0 = pm.Normal("beta0", mu=-3, sigma=1) # Made up by myself
    beta1 = pm.Exponential("beta1", lam=2) # given from the paper
    # linear model
    mu = beta0 + beta1 * x
    p = pm.Deterministic("p", sigmoid(mu), dims="observation")
    # likelihood
    y_obs = pm.Bernoulli("y", logit_p=mu, observed=[0,0,0,0,0], dims="observation")
    # pm.Logistic("y", mu=mu, observed=data['toxicity_event'], dims="observation")
    

In [ ]:
with logistic_regression_model:
    idata = pm.sample_prior_predictive(samples=1000)

In [ ]:
# Define a sigmoid function
def sigmoid(x, beta0, beta1):
    return 1 / (1 + np.exp(-(beta0 + beta1 * x )))

# Extract betas from the trace
beta0 = np.mean(idata.prior['beta0'].values)
beta1 = np.mean(idata.prior['beta1'].values)

# Generate x values
x_values = np.array([0.5, 1, 3, 5, 6])

# Generate y values (sigmoid probabilities)
y_values = sigmoid(x_values, beta0, beta1)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(x_values, y_values, label='Sigmoid curve', marker='o')
plt.xlabel('Doses')
plt.ylabel('Probability of Toxicity Event')
plt.title("prior predictive check")
plt.legend()

In [ ]:
toxicity_observation = np.random.binomial(1, p=true_toxic_prob_s1[0], size=3) # first cohort, first dose
doses_observed = np.ones(3) * doses[0] # doses given to the three participants
data = pd.DataFrame({'doses':doses_observed, 'toxicity_event': toxicity_observation})
coords = {"observation": data.index.values}

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# with pm.Model(coords=coords) as logistic_regression_model:
#     x = pm.ConstantData("doses", data['doses'], dims="observation")
#     # priors
#     beta0 = pm.Normal("beta0",mu=-1, sigma=2) # Made up by myself
#     beta1 = pm.Exponential("beta1", lam=1) # given from the paper
#     # linear model
#     mu = beta0 + beta1 * x
#     p = pm.Deterministic("p", sigmoid(mu), dims="observation")
#     # likelihood
#     y_obs = pm.Bernoulli("y", logit_p=mu, observed=data['toxicity_event'], dims="observation")
#     # pm.Logistic("y", mu=mu, observed=data['toxicity_event'], dims="observation")
    
with pm.Model(coords=coords) as logistic_regression_model:
    x = pm.ConstantData("doses", data['doses'], dims="observation")
    # priors
    beta0 = pm.Normal("beta0",mu=0, sigma=2) # Made up by myself
    beta1 = pm.Exponential("beta1", lam=1) # given from the paper
    # linear model
    mu = beta0 + beta1 * x
    p = pm.Deterministic("p", sigmoid(mu), dims="observation")
    # likelihood
    y_obs = pm.Bernoulli("y", logit_p=mu, observed=data['toxicity_event'], dims="observation")
    # pm.Logistic("y", mu=mu, observed=data['toxicity_event'], dims="observation")
    

In [ ]:
with logistic_regression_model:
    idata = pm.sample_prior_predictive(samples=3000, random_seed=rng)

In [ ]:
# Define a sigmoid function
def sigmoid(x, beta0, beta1):
    return 1 / (1 + np.exp(-(beta0 + beta1 * x )))

# Extract betas from the trace
beta0 = np.mean(idata.prior['beta0'].values)
beta1 = np.mean(idata.prior['beta1'].values)

# Generate x values
x_values = np.linspace(0, max(doses), num=1000)

# Generate y values (sigmoid probabilities)
y_values = sigmoid(x_values, beta0, beta1)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(x_values, y_values, label='Sigmoid curve')
# plt.scatter(data['doses'], data['toxicity_event'], color='red', label='Data')
plt.xlabel('Doses')
plt.ylabel('Probability of Toxicity Event')
plt.title("prior predictive check")
plt.legend()

# Production Test

In [ ]:
def sim_data(relevant_index):
    """ Simulates whether any of the 3 participants have a toxicity event given by the unknown probabilities.

    Args:
        relevant_index (Int): The index associated with the dose being used on the three samples.

    Returns:
        Pandas Dataframe: Contains the doses the three participants took and whether each participant had a toxicity event
    """
    toxicity_observation = np.random.binomial(1, p=true_toxic_prob_s1[relevant_index], size=3) # 3 samples per cohort
    doses_observed = np.ones(3) * doses[relevant_index] # doses given to the three participants
    # store data in a dataframe that will later be concatenated to make one dataframe
    data = pd.DataFrame({'doses':doses_observed, 'toxicity_event': toxicity_observation})
    return data

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def run_model(data):
    """ Creates a logistic regression model using the priors given in the paper
    and the data collected up until this point. 

    Args:
        data (Pandas Dataframe): Same dataframe given by sim_data()

    Returns:
        pymc model: Fitted model based on priors and the current data collected
    """
    coords = {"observation": data.index.values} 
    with pm.Model(coords=coords) as logistic_regression_model:
        x = pm.ConstantData("doses", data['doses'].values, dims="observation")
        # priors
        beta0 = pm.Normal("beta0", mu=-3, sigma=2) # Made up by myself
        beta1 = pm.Exponential("beta1", lam=1) # given from the paper
        # linear model
        mu = beta0 + beta1 * x
        # probabilities
        p = pm.Deterministic("p", sigmoid(mu), dims="observation")
        # likelihood
        y_obs = pm.Bernoulli("y", logit_p=mu, observed=data['toxicity_event'].values, dims="observation") 
    return logistic_regression_model
    
    
def get_next_dose(idata):
    # Extract betas from the trace
    beta0 = np.mean(idata.posterior['beta0'].values)
    beta1 = np.mean(idata.posterior['beta1'].values)
    # store the doses in a numpy array
    x_values = np.array(doses)
    # Generate y values (sigmoid probabilities)
    y_values = sigmoid(beta0 + beta1 * x_values)
    # select the highest dose that falls under the MTD threshold (0.33)
    # if the lowest dose is predicted to be above the threshold, return the next dose INDEX equal to 0 which is a dose of 0.5
    if len(y_values[y_values <= 0.33]) == 0:
        next_dose = 0
    else:
        next_dose = np.argmax(y_values[y_values <= 0.33])
    return next_dose


def run_single_trial():
    """ The setup is as follows:
    1. Choose a starting dose to test on the first three patients. 
    2. Simulate whether each patient has a toxicity event with sim_data().
    3. run the model with the data collected so far. 
    4. Using the posterior estimates of the beta's, predict the highest dose that has a predicted probability of toxicity below 0.33.
    5. Repeat steps 2-4 until all 36 patients have been tested.
    
    numpyro appears to be a faster engine and is able to run the model in a reasonable amount of time. There are other options 
    such as variational inference or even quadratic approximation. However, I am not familiar with these methods.
    """
    # first run of the model with a starting dose of 1 (index of 1)
    data = sim_data(0) 
    logistic_regression_model = run_model(data)
    with logistic_regression_model:
        # idata = pm.sample(500, tune=200, chains=2, random_seed=rng, cores=4, progressbar=False, nuts_sampler='numpyro')
        idata = pm.sampling.jax.sample_numpyro_nuts(800, 200, cores=5, target_accept=0.65, progressbar=False)
    next_dose = get_next_dose(idata)
    
    # We already went through 3 samples out of 36. 36 // 3 = 12 - 1 = 11
    for remaining_trials in range(11):
        new_data = sim_data(next_dose)
        # combine the data collected previously with the new simulated data
        data = pd.concat([data, new_data], axis=0, ignore_index=True)
        # run the model on all the data collected so far
        logistic_regression_model = run_model(data)
        with logistic_regression_model:
            idata = pm.sampling.jax.sample_numpyro_nuts(800, 200, cores=5, target_accept=0.65, progressbar=False)
            # idata = pm.sample(500, tune=200, chains=2, random_seed=rng, cores=4, progressbar=False, nuts_sampler='numpyro')
        next_dose = get_next_dose(idata)
    
    beta0 = np.mean(idata.posterior['beta0'].values)
    beta1 = np.mean(idata.posterior['beta1'].values)
    # store the doses in a numpy array
    x_values = np.array(doses)
    # Generate y values (sigmoid probabilities)
    y_values = sigmoid(beta0 + beta1 * x_values)
    
    return data, y_values

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
total_data = []
sigmoid_probabilities = []
# we should have 1000 simulations but using 350 for now
for sim in tqdm(range(350)):
    data, trial_probabilities = run_single_trial()
    total_data.append(data)    
    sigmoid_probabilities.append(trial_probabilities)

In [ ]:
# with open("state.bin", "wb") as f: # "wb" because we want to write in binary mode
#     pickle.dump(total_data, f)
    
# with open("state.bin", "rb") as f: # "rb" because we want to read in binary mode
#     total_data = pickle.load(f)

In [ ]:
import pickle

with open("sigmoid_probs_pt4.pkl", "wb") as f: # "wb" because we want to write in binary mode
    pickle.dump(sigmoid_probabilities, f)

In [ ]:
for i in range(len(total_data)):
    # add a column that tracks which simulation run is associated with each row
    total_data[i]['sim_run'] = i
    
# combine list of dataframes to one dataframe
total_data = pd.concat(total_data) 

In [ ]:
total_data.to_parquet('finished_simulations_pt4.parquet')

In [ ]:
total_data['doses'][::3].value_counts()

In [ ]:
total_data = pd.read_parquet('finished_simulations.parquet')

In [ ]:
total_data['doses'][::3].value_counts(normalize=True)

In [ ]:
# Extract betas from the trace
beta0 = np.mean(idata.posterior['beta0'].values)
beta1 = np.mean(idata.posterior['beta1'].values)

# Define a sigmoid function
def sigmoid(x, beta0, beta1):
    return 1 / (1 + np.exp(-(beta0 + beta1* x )))

# Generate x values
x_values = np.linspace(0, max(doses), num=1000)

# Generate y values (sigmoid probabilities)
y_values = sigmoid(x_values, beta0, beta1)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(x_values, y_values, label='Sigmoid curve')
plt.scatter(data['doses'], data['toxicity_event'], color='red', label='Data')
plt.scatter(doses, true_toxic_prob_s1, color='black', label='True Probabilities')
plt.xlabel('Doses')
plt.ylabel('Probability of Toxicity Event')
plt.legend()

# Results

60\%

and 45\%

In [ ]:
def get_selected_dose(sigmoid_probs):
    selected_doses = []
    for probs_list in sigmoid_probs:
        if len(probs_list[probs_list <= 0.33]) == 0:
            selected_doses.append(0)
        else:
            selected_doses.append(np.argmax(probs_list[probs_list <= 0.33]))
    return np.array(selected_doses)

In [ ]:
df = pd.read_parquet("gabe_sim_files/S2_finished_simulations.parquet")

In [ ]:
with open("gabe_sim_files/S2_sigmoid_probs.pkl", "rb") as f: # "rb" because we want to read in binary mode
    S2_sigmoid_probs = pickle.load(f)

In [ ]:
S2_sigmoid_probs = np.array(S2_sigmoid_probs)
selected_doses = get_selected_dose(S2_sigmoid_probs)
S2_count_series = pd.Series({dose: np.sum(selected_doses == dose) for dose in [0, 1, 2, 3, 4]})
S2_count_series / S2_count_series.sum()

In [ ]:
with open("gabe_sim_files/sigmoid_probs_pt3.pkl", "rb") as f: # "rb" because we want to read in binary mode
    sigmoid_probs_pt3 = pickle.load(f)
with open("gabe_sim_files/sigmoid_probs_pt4.pkl", "rb") as f: # "rb" because we want to read in binary mode
    sigmoid_probs_pt4 = pickle.load(f)

In [ ]:
S1_sigmoid_probs = np.concatenate((np.array(sigmoid_probs_pt3), np.array(sigmoid_probs_pt4)))
selected_doses = get_selected_dose(S1_sigmoid_probs)
S1_count_series = pd.Series({dose: np.sum(selected_doses == dose) for dose in [0, 1, 2, 3, 4]})
S1_count_series.index = doses
S1_count_series

In [ ]:
np.sum([(y_val <= 0.33)[0].any() for y_val in S1_sigmoid_probs]) / len(S1_sigmoid_probs)

In [ ]:
S1_count_series / S1_count_series.sum()